GPT2-Medium Fine-Tuning With T4 Configuration
---

This notebook is an example on how to create a fine-tuned gpt2-model based on reddit training data from a run on [collection notebook](run_only_fans_collection.ipynb)

This configuration is known to run for a file with 3000 samples in the training file. It will run for 10 epochs distributing load across all available GPUs. Number of batches is auto-calculated.

In [ ]:
import gc

import pandas
import torch
from torch.utils.data import random_split
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import TrainingArguments, Trainer

from src.datasets.reddit_dataset import RedditDataset
from src.tensor_encoding.tensor_encoding import TensorHelper

tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium',
										  bos_token='<|startoftext|>',
										  eos_token='<|endoftext|>',
										  pad_token='<|pad|>')

tokenizer.save_pretrained('./results/tokenizer')

model = GPT2LMHeadModel.from_pretrained('gpt2').cuda()

df = pandas.read_csv("training.csv")

conversations = df['TrainingString']

valid_prompts = []
for conversation in conversations:
	encoded = TensorHelper.encode_and_check(tokenizer, conversation)
	if encoded is not None:
		valid_prompts.append(encoded)

max_length = max([len(tokenizer.encode(prompt)) for prompt in valid_prompts])

model.resize_token_embeddings(len(tokenizer))

dataset = RedditDataset(valid_prompts, tokenizer, max_length=max_length)

train_size = int(0.9 * len(dataset))

train_dataset, eval_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

gc.collect()

torch.cuda.empty_cache()

args = dict(
	output_dir='./results',
	num_train_epochs=10,
	logging_steps=100,
	save_steps=1000,
	# Allow the computer to be smarter than you...
	# per_device_train_batch_size=2,
	# per_device_eval_batch_size=2,
	weight_decay=0.05,
	logging_dir='./logs',
	report_to='none',
	fp16=True,
	auto_find_batch_size=True,
	gradient_accumulation_steps=10,
	learning_rate=1e-4)

training_args = TrainingArguments(**args)

Trainer(model=model, args=training_args, train_dataset=train_dataset,
		eval_dataset=eval_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
															   'attention_mask': torch.stack([f[1] for f in data]),
															   'labels': torch.stack([f[0] for f in data])
															   }).train()